In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import json
from sqlalchemy import create_engine

In [35]:
driver='postgresql'
username='dab_ds23241a_223'
dbname=username # it is the same as the username
password='Bh6KFgcWazMMCPvZ'
server='bronto.ewi.utwente.nl'
port='5432'

# Create an engine instance
alchemy_engine = create_engine(f'{driver}://{username}:{password}@{server}:{port}/{dbname}')

# Connect to PostgreSQL server
db_connection = alchemy_engine.connect()

# Read the data into a_df
incidents_df= pd.read_sql(f"select * from \"project\".\"Incidents_With_Cameras\"", db_connection)

# Check if 'before' and 'after' columns are already dictionaries
if incidents_df['before'].apply(type).eq(dict).all() and incidents_df['after'].apply(type).eq(dict).all():
    print('Columns are already dictionaries.')
else:
    # Convert the 'before' and 'after' columns from JSON strings back to dictionaries if they are not already
    incidents_df['before'] = incidents_df['before'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
    incidents_df['after'] = incidents_df['after'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
# Close the database connection
db_connection.close()

incidents_df.head()

,id,starttijd,type,gedetailleerd_type,primaire_locatie_lengtegraad,primaire_locatie_breedtegraad,segment,before,after
0,RWS02_0000157188_157188,2020-03-01 08:38:28,vehicle_obstruction,emergencyVehicle,4.366420,51.920952,north,{'RWS01_MONIBAS_0041hrl0706ra': 0.022954588858...,{'GEO0K_K_RWSTI357132': 0.16579609461072695}
1,RWS02_0000157197_157197,2020-03-01 10:00:57,vehicle_obstruction,emergencyVehicle,4.432840,51.928864,west-east,{'RWS01_MONIBAS_0201hrr0284ra': 0.094239550154...,{'RWS01_MONIBAS_0201hrr0285ra': 0.065785586736...
2,RWS02_0000157269_157269,2020-03-01 13:21:58,vehicle_obstruction,emergencyVehicle,4.456025,51.864422,east-west,{'RWS01_MONIBAS_0150vwy0557ra': 0.249505010285...,{'RWS01_MONIBAS_0150vwy0553ra': 0.229508095798...
3,RWS03_816372_1,2020-03-01 15:19:57,vehicle_obstruction,brokenDownVehicle,4.538568,51.909523,south,{'RWS01_MONIBAS_0160vwx0193ra': 0.896763594374...,{'RWS01_MONIBAS_0160vwx0202ra': 0.001663796286...
4,RWS02_0000157393_157393,2020-03-01 23:03:29,vehicle_obstruction,emergencyVehicle,4.439620,51.868599,east-west,{'RWS01_MONIBAS_0150vwy0544ra': 0.166980200322...,{'RWS01_MONIBAS_0150vwy0540ra': 0.210197772623...


In [6]:
march_1_7_df = pd.read_csv('camerasData2.csv')
# Create a copy of the DataFrame with only the required columns to avoid SettingWithCopyWarning
march_1_7_df = march_1_7_df[['id_meetlocatie', 'start_meetperiode', 'eind_meetperiode', 'gem_snelheid']].copy()
march_1_7_df['start_meetperiode'] = pd.to_datetime(march_1_7_df['start_meetperiode'])
march_1_7_df['eind_meetperiode'] = pd.to_datetime(march_1_7_df['eind_meetperiode'])

# Replace the negative values with 0 using .loc to ensure the operation is done in place
march_1_7_df.loc[march_1_7_df['gem_snelheid'] < 0, 'gem_snelheid'] = 0

# aggregate instances where id_meetlocatie, start_meetperiode, eind_meetperiode are the same, and take the mean of the gem_snelheid
march_1_7_df = march_1_7_df.groupby(['id_meetlocatie', 'start_meetperiode', 'eind_meetperiode']).agg({'gem_snelheid': 'mean'}).reset_index()
# Extract unique id_meetlocatie values into a list
unique_camera_ids = march_1_7_df['id_meetlocatie'].unique().tolist()

print(unique_camera_ids)

['RWS01_MONIBAS_0151hrl0533ra', 'RWS01_MONIBAS_0151hrl0543ra', 'RWS01_MONIBAS_0151hrl0548ra', 'RWS01_MONIBAS_0151hrl0554ra', 'RWS01_MONIBAS_0151hrl0560ra', 'RWS01_MONIBAS_0151hrl0567ra', 'RWS01_MONIBAS_0151hrl0572ra', 'RWS01_MONICA_00D00F08D446D005000D']


In [44]:
# Filter incidents_df by date
start_date = pd.to_datetime('2020-03-29')
end_date = pd.to_datetime('2020-04-04')
incidents_df = incidents_df[(incidents_df['starttijd'] >= start_date) & (incidents_df['starttijd'] <= end_date)]
print(len(incidents_df))


19


In [42]:

# Heatmap data
heatmap_df = pd.DataFrame(columns=['incident_id', 'x', 't', 'v'])

for index, incident in incidents_df.iterrows():
    incident_time = pd.to_datetime(incident['starttijd'])
    print(f'Incident: {incident.id}')
    print(f'Incident time: {incident_time}')
    start_time = incident_time - pd.Timedelta(hours=1)
    print(f'Start time recording: {start_time}')
    end_time = incident_time + pd.Timedelta(hours=8)
    print(f'End time recording: {end_time}')
    
    # Combine before and after camera IDs into a single list for processing
    camera_ids = list(incident['before'].keys()) + list(incident['after'].keys())
    print(f'Cameras for incidet {incident.id}: {camera_ids}')

    # Check for matching camera IDs in march_1_7_df and that they have speed data
    matching_cameras = march_1_7_df[march_1_7_df['id_meetlocatie'].isin(camera_ids)]
    
    # If there are no matching cameras, skip to the next incident
    if matching_cameras.empty:
        print(f"No matching cameras found for incident {incident['id']}")
        print()
        continue
    # Now, for each camera ID, check if there are records within the time range
    for camera_id, distance in {**incident['before'], **incident['after']}.items():
        camera_speed_data = matching_cameras[(matching_cameras['id_meetlocatie'] == camera_id) &
                                             (matching_cameras['start_meetperiode'] >= start_time) &
                                             (matching_cameras['start_meetperiode'] <= end_time)]
        
        # If there is no speed data for this camera, skip to the next
        if camera_speed_data.empty:
            print(f"No speed data found for camera {camera_id} for incident {incident['id']}")
            continue
        
        # Add rows to heatmap_df with speed data
        for _, speed_data in camera_speed_data.iterrows():
            # Check if the camera is before or after the incident to adjust 'x'
            x_position = 3000 - distance if camera_id in incident['before'] else 3000 + distance
            
            heatmap_df = heatmap_df.append({
                'incident_id': incident['id'],
                'x': x_position,
                't': (speed_data['start_meetperiode'] - start_time).total_seconds() / 3600,
                'v': speed_data['gem_snelheid']
            }, ignore_index=True)

# Display non-empty heatmap_df
if not heatmap_df.empty:
    print(heatmap_df.head())
else:
    print("No data appended to heatmap_df")

print (camera_ids)
# Make sure the 'x' and 't' columns are numeric
heatmap_df['x'] = pd.to_numeric(heatmap_df['x'])
heatmap_df['t'] = pd.to_numeric(heatmap_df['t'])

heatmap_df.head()

Incident: RWS02_0000171458_171458
Incident time: 2020-03-29 15:14:24
Start time recording: 2020-03-29 14:14:24
End time recording: 2020-03-29 23:14:24
Cameras for incidet RWS02_0000171458_171458: ['RWS01_MONIBAS_0201hrr0346ra', 'RWS01_MONIBAS_0201hrr0343ra', 'RWS01_MONIBAS_0201hrr0338ra', 'RWS01_MONIBAS_0201hrr0333ra', 'RWS01_MONIBAS_0201hrr0354ra', 'RWS01_MONIBAS_0201hrr0360ra', 'RWS01_MONIBAS_0201hrr0363ra', 'RWS01_MONIBAS_0201hrr0366ra', 'RWS01_MONIBAS_0201hrr0369ra', 'RWS01_MONIBAS_0201hrr0373ra']
No matching cameras found for incident RWS02_0000171458_171458

Incident: RWS02_0000171529_171529
Incident time: 2020-03-30 03:52:56
Start time recording: 2020-03-30 02:52:56
End time recording: 2020-03-30 11:52:56
Cameras for incidet RWS02_0000171529_171529: ['RWS01_MONIBAS_0160vwx0215ra', 'RWS01_MONIBAS_0160vwx0211ra', 'RWS01_MONICA_10D010033400B8200007', 'RWS01_MONIBAS_0160vwx0202ra', 'RWS01_MONIBAS_0160vwx0219ra', 'GEO0K_K_RWSTI357777']
No matching cameras found for incident RWS02_000

,incident_id,x,t,v


In [46]:
camera_ids =['GEO0B_R_RWSTI356574']

camera_presence = {camera_id: camera_id in march_1_7_df['id_meetlocatie'].unique() for camera_id in camera_ids}
print(camera_presence)

{'GEO0B_R_RWSTI356574': False}
